In [1]:
import numpy as np
import math
import pprint

# 共同住宅の躯体性能の簡易評価法

## I. Orientation Class

### Normal Parts

In [2]:
class OrientationValue():
    
    def __init__( self, v, o ):
        # 'v' is Value and 'o' is Orientation 
        if not ( o == 'D0' or o == 'D90' or o == 'D180' or o == 'D270' or o == 'top' or o == 'bottom' ):
            raise ValueError
        self.v = v
        self.o = o
        
    def __str__( self ):
        return {
            'D0'     : 'D0     : ',
            'D90'    : 'D90    : ',
            'D180'   : 'D180   : ',
            'D270'   : 'D270   : ',
            'top'    : 'top    : ',
            'bottom' : 'bottom : ',
               }[self.o] + str(self.v)

In [3]:
class HorizontalOrientation():
    
    def __init__( self, d0, d90, d180, d270 ):
        self.d0   = d0
        self.d90  = d90
        self.d180 = d180
        self.d270 = d270
        
    def getMembers(self):
        return [ self.d0, self.d90, self.d180, self.d270 ]
    
    def get_dict(self):
        
        return {
            'D0'   : self.d0.v,
            'D90'  : self.d90.v,
            'D180' : self.d180.v,
            'D270' : self.d270.v,
        }
    
    def __str__(self):
        return 'd0: ' + str(self.d0) + ', d90: ' + str(self.d90) + ', d180: ' + str(self.d180) + ', d270: ' + str(self.d270)

#### Example

In [4]:
try:
    OrientationValue( 0.12, 'D45')
except ValueError:
    print('error')

error


In [5]:
print( OrientationValue( 0.12, 'D90' ) )

D90    : 0.12


### Heat Bridges

In [6]:
class OrientationValueHB():
    def __init__( self, v, o1, o2 ):
        
        # 'v' is Value, 'o1' is Orientation, 'o2' is Orientation 
        def ValueCheck( o ):
            if not ( o == 'D0' or o == 'D90' or o == 'D180' or o == 'D270' or o == 'top' or o == 'bottom' ):
                raise ValueError
        ValueCheck( o1 )
        ValueCheck( o2 )
        self.v = v
        self.o1 = o1
        self.o2 = o2
    
    def get_info(self):
        
        return {
            'orientation1' : self.o1,
            'orientation2' : self.o2,
            'value'        : self.v,
        }
    
    def __str__( self ):
        return {
            'D0'     : 'D0     + ',
            'D90'    : 'D90    + ',
            'D180'   : 'D180   + ',
            'D270'   : 'D270   + ',
            'top'    : 'top    + ',
            'bottom' : 'bottom + ',
               }[self.o1] + {
            'D0'     : 'D0     : ',
            'D90'    : 'D90    : ',
            'D180'   : 'D180   : ',
            'D270'   : 'D270   : ',
            'top'    : 'top    : ',
            'bottom' : 'bottom : ',
               }[self.o2] + str(self.v)

In [7]:
try:
    OrientationValueHB( 0.12, 'D45', 'D90' )
except ValueError:
    print('error')

error


In [8]:
try:
    OrientationValueHB( 0.12, 'D90', 'D45')
except ValueError:
    print('error')

error


In [9]:
print( OrientationValueHB( 0.12, 'top', 'D90' ) )

top    + D90    : 0.12


## II. Input Data

### 1) 住棟全体

| No. | 入力項目 | 単位 | 記号 | 備考 |
| --- | --- | --- | --- | --- |
| G1 | 地域の区分 | - | - | 1～8地域 |
| G2 | 建物の構造 | - | - | RC, S, W |
| G3 | 建物の階数 | 階 | $N$ | - |
| G4 | 建物全体の高さ | m | $H_{total}$ | - |
| G5 | 主開口部の方位 | - | - | 8方位 |

````
common = {
    'region'                 : 6,    # 地域の区分
    'structure'              : 'RC', # 建物の構造
    'levels_number'          : 3,    # 建物の階数
    'height'                 : 9.1,  # 建物の高さ(m)
    'opening_main_direction' : 'S',  # 主開口部の方位
}
````

### 2) 専有部全体

| No. | 入力項目 | 単位 | 記号 | 備考 |
| --- | --- | --- | --- | --- |
| P9 | 外壁の仕様 熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P10 | 外壁の仕様 断熱位置 | - | | inside=内断熱, outside=外断熱, non=無断熱 から選択 |
| P11 | 外壁の仕様 断熱補強の仕様 | - | | spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択 |
| P12 | 屋根の仕様 熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P13 | 屋根の仕様 断熱位置 | - | | inside=内断熱, outside=外断熱, non=無断熱 から選択 |
| P14 | 屋根の仕様 断熱補強の仕様 | - | | spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択 |
| P15 | 最下階床の仕様 熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P16 | 最下階床の仕様 断熱位置 | - | | inside=内断熱, outside=外断熱, non=無断熱 から選択 |
| P17 | 最下階床の仕様 断熱補強の仕様 | - | | spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択 |
| P18 | 界壁の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P19 | 界床の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P20 | 窓の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| P21 | 窓の日射熱取得率(η値) | - | | |

````
private_area = {
    'wall'                : { # 外壁
        'U_value' : 0.785,    # 熱貫流率(U値)(W/m2K)
        'ins_pos' : 'inside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        'HB_spec' : 'spec1'   # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    'roof'                : {  # 屋根
        'U_value' : 0.417,     # 熱貫流率(U値)(W/m2K)
        'ins_pos' : 'outside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        'HB_spec' : 'spec1'    # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    'floor'               : {  # 最下階床
        'U_value' : 0.507,     # 熱貫流率(U値)(W/m2K)
        'ins_pos' : 'outside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
        'HB_spec' : 'null'     # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
    },
    'inside_wall'  : {         # 界壁
        'U_value' : 2.34,      # 熱貫流率(U値)(W/m2K)
    },
    'inside_floor' : {         # 界床
        'U_value' : 1.796,     # 熱貫流率(U値)(W/m2K)
    },
    'window' : {               # 窓
        'U_value'   : 4.65,    # 熱貫流率(U値)(W/m2K)
        'eta_value' : 0.79     # 日射熱取得率(η値)    
    },
    'private_floors' : [private_floor],
}
````

| No. | 入力項目 | 単位 | 記号 | 備考 |
| --- | --- | --- | --- | --- |
| P1 | 各階の住戸数 | 戸 | $n$ | 階ごとに入力する |
| P2 | 各階の床面積の合計 | m<sup>2</sup>| $A_{f,total} $ | 階ごとに入力する |
| P3 | 各階の外周長 外気と接する部分 | m | | 階ごとに入力する |
| P4 | 各階の外周長 空調された共用部や非住宅部と接する部分 | m | | 階ごとに入力する |
| P5 | 各階の外周長 住戸と接する部分 | m | | 階ごとに入力する (住棟を複数の方位に分けて計算する場合のみ入力) |
| P6 | 各階の窓面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| P7 | 各階(最上階を除く)の屋根面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| P8 | 各階(最下階を除く)の外気床の面積の合計 | m<sup>2</sup> | | 階ごとに入力する |

````
private_floor = {
    'house_number'        : 4,                # 各階の住戸数
    'floor_area'          : 162.64, # 各階の床面積の合計(m2)
    'peri_length_outside' : [66.56, 66.56, 66.56],    # 各階の外周長 外気と接する部分(m)
    'PeriLengthInside'    : [0.0, 0.0, 0.0],          # 各階の外周長 空調された共用部や非住宅部と接する部分(m)
    'PeriLengthPrivate'   : [0.0, 0.0, 0.0],          # 各階の外周長 住戸と接する部分(住棟を複数の方位に分けて計算する場合のみ)(m)
    'window_area'         : [30.48, 30.64, 30.64],    # 各階の窓面積の合計(m2)
    'roof_area'           : [0.0, 0.0, None],         # 各階(最上階を除く)の屋根面積の合計(m2)
    'outside_floor_area'  : [None, 0.0, 0.0],         # 各階(最下階を除く)の外気床の面積の合計(m2)
}
````

### 3) 共用部

| No. | 入力項目 | 単位 | 記号 | 備考 |
| --- | --- | --- | --- | --- |
| C1 | 各階の共用部・非住宅部の床面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| C2 | 各階(最上階を除く)の屋根面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| C3 | 各階(最下階を除く)の外気床の面積の合計 | m<sup>2</sup> | | 階ごとに入力する |
| C4 | 外壁の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C5 | 界床の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C6 | 外気床の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C7 | 窓の熱貫流率(U値) | W/m<sup>2</sup>K | | |
| C8 | 窓の日射熱取得率(η値) | - | |  |

````
d_public_area = {
    'FloorArea'          : [68.82, 59.7, 59.7], # 各階の共用部・非住宅部の床面積の合計(m2)
    'RoofArea'           : [9.125, 0.0, None],  # 各階(最上階を除く)の屋根面積の合計(m2)
    'OutsideFloorArea'   : [None, 0.0, 0.0],    # 各階(最下階を除く)の外気床の面積の合計(m2)
    'WallUValue'         : 0.00,                # 外壁の熱貫流率(U値)(W/m2K)
    'InsideFloorUValue'  : 0.00,                # 界床の熱貫流率(U値)(W/m2K)
    'OutsideFloorUValue' : 0.00,                # 外気床の熱貫流率(U値)(W/m2K)
    'WindowUValue'       : 0.00,                # 窓の熱貫流率(U値)(W/m2K)
    'WindowEtaValue'     : 0.00                 # 窓の日射熱取得率(η値)
}
````

````
d = {
    "common"       : common,
    "private_area" : private_area,
    "public_area"  : public_area
}
````

## IV. Functions

### 1. 専有部における各階の面積・長さ

### 1.1 階高

$$
H_i = \frac{H_{total}}{N}
$$

$H_i$: 階$i$の階高, m  
$H_{total}$: 建物全体の高さ, m  
$N$: 建物の階数  

In [10]:
def get_height(height, levels):
    
    return height / levels

### 1.2 屋根面積

専有部における階$i$の屋根面積の合計は最上階とそれ以外の階に分けて求める。最上階以外は入力値とし、最上階は床面積の合計に等しいものとする。なお、最上階以外に屋根がある場合は、その階の主方位側かつ各住戸に均等に位置するものとする。

$$
A_{r,total,i} = \left\{
    \begin{array}{ll}
        A_{r,total,i,input} & (i \ne N) \\
        A_{f,total,i} & (i = N)
    \end{array}
\right.
$$

$N$: 建物の階数(階)  
$A_{r,h,total,i}$: 専有部(住戸部)における階$i$の屋根面積の合計, m<sup>2</sup>

In [11]:
def get_roof_area_total(roof_area_total_input, A_f_total, level, total_level_number):
    
    return roof_area_total_input if level != total_level_number - 1 else A_f_total

### 1.3 外気に接する床の面積

階$i$における専有部の外気に接する床の面積の合計は、1階(最下階)は0とし、1階以外は入力値とする。  
なお、最下階以外に外気に接する床がある場合は、その階の主方位側かつ各住戸に均等に位置するものとする。

$$
A_{of,total,i} = \left\{
    \begin{array}{ll}
        0 & (i = 1) \\
        A_{of,total,i,input} & (i \ne 1)
    \end{array}
\right.
$$

$A_{of,total,i}$: 階$i$における専有部の外気に接する床の面積の合計(m<sup>2</sup>)

In [12]:
def get_outside_floor_area_total(A_of_total_input, level):
    return 0.0 if level == 0 else A_of_total_input

### 1.4 長辺および短辺の長さ

階$i$における専有部全体の平面形状は矩形を想定し、その長辺が主方位(主方位+180°)側にあるもとの仮定し、長辺と短辺の長さの合計は次式を連立して求めることとする。

$$ l_{long,total,i} \ge l_{short,total,i} $$
$$ l_{long,total,i} \times l_{short,total,i} = A_{f,total,i} $$
$$ 2 \times ( l_{long,total,i} + l_{short,total,i} ) = L_i $$

これらを連立すると、

$$ l_{long,total,i} = \frac{ \frac{L_i}{2} + \sqrt{ (\frac{L_i}{2})^2 - 4 \times A_{f,total,i} } }{2} $$

$$ l_{short,total,i} = \frac{ \frac{L_i}{2} - \sqrt{ (\frac{L_i}{2})^2 - 4 \times A_{f,total_i} } }{2} $$

となる。

$ A_{f,total,i} $: 階$i$における専有部の床面積の合計(m<sup>2</sup>)  
$ l_{long,total,i} $: 階$i$における専有部全体の長辺の長さの合計(m)  
$ l_{short,total,i} $: 階$i$における専有部全体の短辺の長さの合計(m)  
$ L_i$: 階$i$における専有部全体の外皮の周長(m)

In [13]:
def get_l_long_total(L, A_f_total):
    return ( (L/2) + math.sqrt( (L/2)**2 - 4 * A_f_total ) )/2

In [14]:
def get_l_short_total(L, A_f_total):
    return ( (L/2) - math.sqrt( (L/2)**2 - 4 * A_f_total ) )/2

### 1.5 窓面積

階$i$における主方位から時計回りに0°、90°、180°及び270°の方位の窓面積の合計は次式により算出する。なお、主方位を$D0$とし、方位の角度は上から見て時計回りに増加するものとし、$D90$で主方位+90度、$D180$で主方位+180度、$D270$で主方位+270度とする。（以下、同じ）。

$$
A_{win,total,D0,i} = A_{win,total,i} \times ( 1 - \frac{0.24}{n_i} ) \times 0.64
$$

$$
A_{win,total,D90,i} = A_{win,total,i} \times \frac{0.12}{n_i}
$$

$$
A_{win,total,D180,i} = A_{win,total,i} \times ( 1 - \frac{0.24}{n_i} ) \times 0.36
$$

$$
A_{win,total,D270,i} = A_{win,total,i} \times \frac{0.12}{n_i}
$$

$A_{win,total,i}$: 階$i$における窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D0,i}$: 階$i$における主方位から時計回りに0°の方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D90,i}$: 階$i$における主方位から時計回りに90°の方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D180,i}$: 階$i$における主方位から時計回りに180°の方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D270,i}$: 階$i$における主方位から時計回りに270°の方位の窓面積の合計(m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [15]:
def get_windows_area_total(A_win_total, n):
    return HorizontalOrientation(
        OrientationValue( A_win_total * (1-0.24/n) * 0.64, 'D0'   ),
        OrientationValue( A_win_total * 0.12/n,            'D90'  ),
        OrientationValue( A_win_total * (1-0.24/n) * 0.36, 'D180' ),
        OrientationValue( A_win_total * 0.12/n,            'D270' )
    )

### 2. 専有部における各階の各住戸の面積・長さ

### 2.1 屋根の面積

$$
A_{r,i,j} = \frac{A_{r,total,i}}{n_i}
$$

$A_{r,i,j}$: 階$i$における住戸$j$の屋根面積 (m<sup>2</sup>)  
$A_{r,total,i}$: 階$i$における屋根面積の合計 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数 (戸)

In [16]:
def get_roof_area(A_r_total, n):
    return A_r_total / n

### 2.2 窓の面積

階$i$の住戸$j$における主方位から時計回りに0°、90°、180°及び270°の方位の窓面積$A_{win,D0,i,j}$, $A_{win,D90,i,j}$, $A_{win,D180,i,j}$, $A_{win,D270,i,j}$は、住戸位置ごとに次式により算出する。

$$
A_{win,D0,i,j} = \frac{ A_{win,total,D0,i} } { n_i }
$$

$$
A_{win,D90,i,j} = \left\{
    \begin{array}{ll}
        A_{win,total,D90,i} & (j=1) \\
        0 & (j \ne 1)
    \end{array}
\right.
$$

$$
A_{win,D180,i,j} = \frac{ A_{win,total,D180,i} } { n_i }
$$

$$
A_{win,D270,i,j} = \left\{
    \begin{array}{ll}
        A_{win,total,D270,i} & (j=n) \\
        0 & (j \ne n)
    \end{array}
\right.
$$

$A_{win,D0,i,j}$: 階$i$の住戸$j$における主方位から時計回りに0°の方位の窓面積(m<sup>2</sup>)  
$A_{win,D90,i,j}$: 階$i$の住戸$j$における主方位から時計回りに90°の方位の窓面積(m<sup>2</sup>)  
$A_{win,D180,i,j}$: 階$i$の住戸$j$における主方位から時計回りに180°の方位の窓面積(m<sup>2</sup>)  
$A_{win,D270,i,j}$: 階$i$の住戸$j$における主方位から時計回りに270°の方位の窓面積(m<sup>2</sup>)  
$A_{win,total,D0,i}$: 階$i$における主方位から時計回りに0°の方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D90,i}$: 階$i$における主方位から時計回りに90°の方位の窓面積の合計(m<sup>2</sup>)  
$A_{win,total,D180,i}$: 階$i$における主方位から時計回りに180°の方位の窓面積の合計(m<sup>2</sup>)   
$A_{win,total,D270,i}$: 階$i$における主方位から時計回りに270°の方位の窓面積の合計(m<sup>2</sup>)  

In [17]:
def get_windows_area(A_win_total, house_number, houseindex):
    return HorizontalOrientation(
        OrientationValue( A_win_total.d0.v / house_number,                             'D0'   ),
        OrientationValue( A_win_total.d90.v if houseindex == 0 else 0,                'D90'  ),
        OrientationValue( A_win_total.d180.v / house_number,                           'D180' ),
        OrientationValue( A_win_total.d270.v if houseindex == house_number - 1 else 0, 'D270' )
    )

### 2.3 玄関ドア面積

階$i$の住戸$j$における主方位から時計回りに0°、90°、180°及び270°の方位の各方位における玄関ドア面積$A_{door,D0,i,j}$, $A_{door,D90,i,j}$, $A_{door,D180,i,j}$, $A_{door,D270,i,j}$ (m<sup>2</sup>)は次式により算出する。

$$
A_{door,D0,i,j} = 0
$$

$$
A_{door,D90,i,j} = 0
$$

$$
A_{door,D180,i,j} = 1.6
$$

$$
A_{door,D270,i,j} = 0
$$

$A_{door,D0,i,j}$: 階$i$の住戸$j$における主方位から時計回りに0°の方位の玄関ドア面積(m<sup>2</sup>)  
$A_{door,D90,i,j}$: 階$i$の住戸$j$における主方位から時計回りに90°の方位の玄関ドア面積(m<sup>2</sup>)  
$A_{door,D180,i,j}$: 階$i$の住戸$j$における主方位から時計回りに180°の方位の玄関ドア面積(m<sup>2</sup>)  
$A_{door,D270,i,j}$: 階$i$の住戸$j$における主方位から時計回りに270°の方位の玄関ドア面積(m<sup>2</sup>)  

In [18]:
def get_doors_area():
    return HorizontalOrientation(
        OrientationValue( 0.0, 'D0'   ),
        OrientationValue( 0.0, 'D90'  ),
        OrientationValue( 1.6, 'D180' ),
        OrientationValue( 0.0, 'D270' )
    ) 

### 2.4 外壁の面積

階$i$の住戸$j$における主方位から時計回りに0°、90°、180°及び270°の方位の各方位における外壁面積$A_{ow,D0,i,j}$, $A_{ow,D90,i,j}$, $A_{ow,D180,i,j}$, $A_{ow,D270,i,j}$は、住戸位置ごとに次式により算出する。なお、主方位+180°に玄関前を想定した窪み(幅1m、奥行き1m)を想定する。

$$
A_{ow,D0,i,j} = \frac{ l_{long,total,i} }{ n_i } \times H_i -  A_{win,D0,i,j} - A_{door,D0,i,j}
$$

$$
A_{ow,D90,i,j} = \left\{
    \begin{array}{ll}
        ( l_{short,total,i} + 1 ) \times H_i - A_{win,D90,i,j} - A_{door,D90,i,j} & (j=1) \\
        1 \times H_i - A_{win,D90,i} - A_{door,D90,i} & (j \ne 1)
    \end{array}
\right.
$$

$$
A_{ow,D180,i,j} = \frac{ l_{long,total,i} }{ n_i } \times H_i -  A_{win,D180,i,j} - A_{door,D180,i,j}
$$

$$
A_{ow,h,k,D270} = \left\{
    \begin{array}{ll}
        ( l_{short,total} + 1 ) \times H - A_{win,h,k,D270} - A_{door,h,D270} & (k=n) \\
        1 \times H - A_{win,h,k,D270} - A_{door,h,D270} & (k \ne n)
    \end{array}
\right.
$$

$A_{ow,D0,i,j}$: 階$i$の住戸$j$における主方位から時計回りに0°の方位の外壁面積(m<sup>2</sup>)  
$A_{ow,D90,i,j}$: 階$i$の住戸$j$における主方位から時計回りに90°の方位の外壁面積(m<sup>2</sup>)  
$A_{ow,D180,i,j}$: 階$i$の住戸$j$における主方位から時計回りに180°の方位の外壁面積(m<sup>2</sup>)  
$A_{ow,D270,i,j}$: 階$i$の住戸$j$における主方位から時計回りに270°の方位の外壁面積(m<sup>2</sup>)  
$l_{long,total,i}$: 階$i$における専有部全体の長辺の長さ (m)  
$l_{short,total,i}$: 階$i$における専有部全体の短辺の長さ (m)  
$n_i$: 階$i$における住戸数 (戸)  
$H_i$: 階$i$の階高 (m)  
$A_{win,D0,i,j}$: 階$i$の住戸$j$における主方位から時計回りに0°の方位の窓面積 (m<sup>2</sup>)  
$A_{win,D90,i,j}$: 階$i$の住戸$j$における主方位から時計回りに90°の方位の窓面積 (m<sup>2</sup>)  
$A_{win,D180,i,j}$: 階$i$の住戸$j$における主方位から時計回りに180°の方位の窓面積 (m<sup>2</sup>)  
$A_{win,D270,i,j}$: 階$i$の住戸$j$における主方位から時計回りに270°の方位の窓面積 (m<sup>2</sup>)  
$A_{door,D0,i,j}$: 階$i$の住戸$j$における主方位から時計回りに0°の方位の玄関ドア面積 (m<sup>2</sup>)  
$A_{door,D90,i,j}$: 階$i$の住戸$j$における主方位から時計回りに90°の方位の玄関ドア面積 (m<sup>2</sup>)  
$A_{door,D180,i,j}$: 階$i$の住戸$j$における主方位から時計回りに180°の方位の玄関ドア面積 (m<sup>2</sup>)  
$A_{door,D270,i,j}$: 階$i$の住戸$j$における主方位から時計回りに270°の方位の玄関ドア面積 (m<sup>2</sup>)  

In [19]:
def get_outside_walls_area( l_long_total, l_short_total, H, A_win, A_door, house_number, houseIndex ):
    return HorizontalOrientation(
        OrientationValue( l_long_total / house_number * H                                  - A_win.d0.v   - A_door.d0.v,   'D0' ),
        OrientationValue( (l_short_total if houseIndex == 0 else 0.0 + 1) * H              - A_win.d90.v  - A_door.d90.v,  'D90' ),
        OrientationValue( l_long_total / house_number * H                                  - A_win.d180.v - A_door.d180.v, 'D180' ),
        OrientationValue( (l_short_total if houseIndex == house_number-1 else 0.0 + 1) * H - A_win.d270.v - A_door.d270.v, 'D270' )
    )

### 2.5 外気に接する床の面積

階$i$の住戸$j$における外気に接する床の面積$A_{of,i,j}$(m<sup>2</sup>)は次式により算出する。なお、最下階($i=1$)の場合は必ず$A_{of,total,i} = 0$のため、$A_{of,i,j}=0$である。

$$
A_{of,i,j} = \frac{ A_{of,total,i} }{ n_i }
$$

$A_{of,i,j}$: 階$i$の住戸$j$における外気に接する床の面積 (m<sup>2</sup>)  
$A_{of,total,i}$: 階$i$における専有部の外気に接する床の面積の合計 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数 (戸)

In [20]:
def get_outside_floor_area( A_of_total, n ):
    return A_of_total / n

### 2.6 界壁の面積

階$i$の住戸$j$の界壁面積$A_{iw,i,j}$(m<sup>2</sup>)は次式により算出する。

$ n_i = 1 $ (階$i$に1住戸しかない場合)の場合、

$$
A_{iw,i,j} = 0.0
$$

$ n_i > 2 $ の場合、

$$
A_{iw,i,j} = \left\{
    \begin{array}{ll}
        l_{short,total,i} \times H_i & ( j = 1 ) \\
        l_{short,total,i} \times H_i \times 2 & ( (j \ne 1) \  and \  (j \ne n_i) ) \\
        l_{short,total,i} \times H_i & ( j = n_i )
    \end{array}
\right.
$$

$n_i$: 階$i$における住戸数(戸)  
$A_{iw,i,j}$: 階$i$の住戸$j$における界壁の面積 (m<sup>2</sup>)  
$l_{short,total,i}$: 階$i$における専有部全体の短辺の長さ (m)  
$H_i$: 階$i$の階高 (m)

In [21]:
def get_inside_wall_area(l_short_total, H, n, idx):
    if idx == 0:
        return l_short_total * H
    elif idx == n - 1:
        return l_short_total * H
    else:
        return l_short_total * H * 2

### 2.7 界床の面積

階$i$の住戸$j$における界床の面積$A_{if,i,j}$ (m<sup>2</sup>) は、次式により算出する。

$$
A_{if,i,j} = \left\{
    \begin{array}{ll}
        0 & (i=1)\\
        \frac{ A_{f,total,i} }{ n_i } - A_{of,i,j} & ( i \ge 2 )
    \end{array}
\right.
$$

$A_{if,i,j}$: 階$i$の住戸$j$における界床面積 (m<sup>2</sup>)  
$A_{f,total,i}$: 階$i$の専有部の床面積の合計 (m<sup>2</sup>)  
$A_{of,i,j}$: 階$i$の住戸$j$における外気に接する床の面積 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [22]:
def get_inside_floor_area(A_f_total, A_of, n, level):
    if level == 0:
        return 0.0
    else:
        return A_f_total / n - A_of

### 2.8 界天井の面積

階$i$の住戸$j$における界天井の面積$A_{ic,i,j}$ (m<sup>2</sup>) は、次式により算出する。

$$
A_{ic,i,j} = \left\{
    \begin{array}{ll}
        0 & (i=N)\\
        \frac{ A_{f,total,i} }{ n_i } - A_{r,i} & ( i < N )
    \end{array}
\right.
$$

$N$: 建物の階数(階)  
$A_{ic,i,j}$: 階$i$の住戸$j$における界天井の面積 (m<sup>2</sup>)  
$A_{f,total,i}$: 階$i$の専有部の床面積の合計 (m<sup>2</sup>)  
$A_{r,i}$: 階$i$の住戸$j$における外気に接する屋根の面積 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [23]:
def get_inside_ceiling_area( A_f_total, A_r, n, level, levelNumber ):
    if level == levelNumber - 1:
        return 0.0
    else:
        return A_f_total/n - A_r

### 2.9 床下に面する床面積

階$i$の住戸$j$における床下に面する床面積$A_{csf,i,j}$ (m<sup>2</sup>) は、次式により算出する。

$$
A_{csf,i,j} = \left\{
    \begin{array}{ll}
        \frac{ A_{f,total,i} }{ n_i } & (i=1)\\
        0 & ( i \ge 2 )
    \end{array}
\right.
$$

$A_{csf,i,j}$: 階$i$の住戸$j$における床下に面する床面積 (m<sup>2</sup>)  
$A_{f,total,i}$: 階$i$の専有部の床面積の合計 (m<sup>2</sup>)  
$n_i$: 階$i$における住戸数(戸)

In [24]:
def get_crawl_space_floor_area(A_f_total, n, level):
    if level > 2:
        return 0.0
    else:
        return A_f_total / n

### 3. 各住戸の熱橋長さ

![HeatBridge](HeatBridgeType.png)

各階における各住戸の構造熱橋部の長さ

| 種類(修正版)<br>(記号) | 種類<br>(記号) | 境界の数<br>(屋外/屋内/床下) | 熱橋の方向 | 主な発生場所 |
| --- | --- | --- | --- | --- |
| $ \psi_{1v,w-w} $  | $ \psi_{1v} $    | 3 / 1 / - | 鉛直 | 外壁同士の接合部 |
| $ \psi_{1h,w-r} $  | $ \psi_{1h,r} $  | 3 / 1 / - | 水平 | 外壁と屋根の接合部 |
| $ \psi_{1h,w-f} $  | $ \psi_{1h,f} $  | 3 / 1 / - | 水平 | 外壁と外気床の接合部 |
| $ \psi_{2v,w} $    | $ \psi_{2v} $    | 2 / 2 / - | 鉛直 | 外壁と間仕切壁の接合部（住戸長辺≧20mの場合は追加） |
| $ \psi_{2h,r} $    | $ \psi_{2h,r} $  | 2 / 2 / - | 水平 | 屋根と界壁の接合部（住戸長辺≧20mの場合は追加） |
| $ \psi_{2h,w} $    | $ \psi_{2h,f1} $ | 2 / 2 / - | 水平 | 外壁と界床の接合部 |
| $ \psi_{2h,f} $  | $ \psi_{2h,f2} $ | 2 / 2 / - | 水平 | 外気床と界壁の接合部 |
| $ \psi_{3v,w-w} $  | $ \psi_{3v} $    | 1 / 3 / - | 鉛直 | 外壁同士の接合部 |
| $ \psi_{3h,w-r} $  | $ \psi_{3h,r} $  | 1 / 3 / - | 水平 | 外壁と中間階屋根の接合部 |
| $ \psi_{3h,w-f} $  | $ \psi_{3h,f} $  | 1 / 3 / - | 水平 | 外壁と外気床の接合部 |
| $ \psi_{1h,w-uf} $ | $ \psi_{4h,uf} $ | 2 / 1 / 1 | 水平 | 外壁と最下階床の接合部 |
| $ \psi_{2h,uf} $   | $ \psi_{5h,uf} $ | - / 2 / 2 | 水平 | 界壁と最下階床の接合部 |


### 3.1 熱橋長さ$l_{\psi 1 v,w-w}$

階$i$における住戸$j$の$\psi_{1v,i,j}$の熱橋長さは、住戸位置ごとに下式により算出する。なお、方位+180に玄関前を想定した窪み(幅1m、奥行き1m)を想定する（以下、同じ）。

$$
l_{\psi 1v,D0-D90,i,j} = \left \{
    \begin{array}{ll}
        H_i & ( j = 1 ) \\
        0 & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1v,D90-D180,i,j} = \left \{
    \begin{array}{ll}
        H_i \times 2 & ( j = 1 ) \\
        H_i & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1v,D180-D270,i,j} = \left \{
    \begin{array}{ll}
        H_i \times 2 & ( j = n ) \\
        H_i & ( j \ne n ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1v,D270-D0,i,j} = \left \{
    \begin{array}{ll}
        H_i & ( j = n ) \\
        0 & ( j \ne n ) \\
    \end{array}
\right.
$$

In [25]:
def get_l_psi_1v_ww( H, n, idx ):
    return [
        OrientationValueHB( H     if idx == 0     else 0.0, 'D0',   'D90'  ),
        OrientationValueHB( H * 2 if idx == 0     else H,   'D90',  'D180' ),
        OrientationValueHB( H * 2 if idx == n - 1 else H,   'D180', 'D270' ),
        OrientationValueHB( H     if idx == n - 1 else 0.0, 'D270', 'D0'   )
    ]

### 3.2 熱橋長さ$l_{\psi 1h,w-r}$

#### i) 最上階以外($i \ne N$)の場合でかつ屋根がある($A_{r,total,i} \gt 0$)場合

$$
l_{\psi 1h,w-r,top-D0,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1h,w-r,top-D90,i,j} = \left \{
    \begin{array}{ll}
        \frac{ A_{r,total,i} }{ l_{long,total,i} } & ( j = 1 ) \\
        0 & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1h,w-r,top-D180,i,j} = 0
$$

$$
l_{\psi 1h,w-r,top-D270,i,j} = \left \{
    \begin{array}{ll}
        \frac{ A_{r,total,i} }{ l_{long,total,i} } & ( j = n_i ) \\
        0 & ( j \ne n_i ) \\
    \end{array}
\right.
$$

#### ii) 最上階以外($i \ne N_i$)の場合でかつ屋根がない($A_{r,total,i} = 0$)場合

$$
l_{\psi 1h,w-r,top-D0,i,j} = 0
$$

$$
l_{\psi 1h,w-r,top-D90,i,j} = 0
$$

$$
l_{\psi 1h,w-r,top-D180,i,j} = 0
$$

$$
l_{\psi 1h,w-r,top-D270,i,j} = 0
$$

#### iii) 最上階($i = N$)の場合

$$
l_{\psi 1h,w-r,top-D0,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1h,w-r,top-D90,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} + 1 & ( j = 1 ) \\
        1 & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1h,w-r,top-D180,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1h,w-r,top-D270,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} + 1 & ( j = n ) \\
        1 & ( j \ne n ) \\
    \end{array}
\right.
$$

In [26]:
def get_l_psi_1h_wr( l_long_total, l_short_total, A_r_total, n, idx, level, levelNumber ):
    if level == levelNumber:
        return [
            OrientationValueHB( l_long_total / n,                           'top', 'D0'   ),
            OrientationValueHB( l_short_total + 1.0 if idx == 0 else 1.0,   'top', 'D90'  ),
            OrientationValueHB( l_long_total / n,                           'top', 'D180' ),
            OrientationValueHB( l_short_total + 1.0 if idx == n-1 else 1.0, 'top', 'D270' ),
        ]
    else:
        if A_r_total == 0:
            return [
                OrientationValueHB( 0.0, 'top', 'D0'   ),
                OrientationValueHB( 0.0, 'top', 'D90'  ),
                OrientationValueHB( 0.0, 'top', 'D180' ),
                OrientationValueHB( 0.0, 'top', 'D270' ),
            ]
        else:
            return[
                OrientationValueHB( l_long_total / n,                                  'top', 'D0'   ),
                OrientationValueHB( A_r_total / l_long_total if idx == 0 else 0.0,     'top', 'D90'  ),
                OrientationValueHB( 0.0,                                               'top', 'D180' ),
                OrientationValueHB( A_r_total / l_long_total if idx == n - 1 else 0.0, 'top', 'D270' ),
            ]        

### 3.3 熱橋長さ$l_{\psi 1h,w-f}$

#### i) 外気床がある($A_{of,total,i} \gt 0$)場合

$$
l_{\psi 1h,w-f,bottom-D0,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1h,w-f,bottom-D90} = \left \{
    \begin{array}{ll}
        \frac{ A_{of,total,i} }{ l_{long,total,i} } & ( j = 1 ) \\
        0 & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 1h,w-f,bottom-D180,i,j} = 0
$$

$$
l_{\psi 1h,w-f,bottom-D270,i,j} = \left \{
    \begin{array}{ll}
        \frac{ A_{of,total,i} }{ l_{long,total,i} } & ( j = n ) \\
        0 & ( j \ne n ) \\
    \end{array}
\right.
$$

#### ii) 外気床がない($A_{of,total,i} = 0$)場合

$$
l_{\psi 1h,w-f,bottom-D0,i,j} = 0
$$

$$
l_{\psi 1h,w-f,bottom-D90,i,j} = 0
$$

$$
l_{\psi 1h,w-f,bottom-D180,i,j} = 0
$$

$$
l_{\psi 1h,w-f,bottom-D270,i,j} = 0
$$

In [27]:
def get_l_psi_1h_wf( l_long_total, A_of_total, n, idx ):
    if A_of_total > 0:
        return [
            OrientationValueHB( l_long_total / n,                                   'bottom', 'D0'   ),
            OrientationValueHB( A_of_total / l_long_total if idx == 0 else 0.0,     'bottom', 'D90'  ),
            OrientationValueHB( 0.0,                                                'bottom', 'D180' ),
            OrientationValueHB( A_of_total / l_long_total if idx == n - 1 else 0.0, 'bottom', 'D270' ),
        ]
    else:
        return [
            OrientationValueHB( 0.0, 'bottom', 'D0'   ),
            OrientationValueHB( 0.0, 'bottom', 'D90'  ),
            OrientationValueHB( 0.0, 'bottom', 'D180' ),
            OrientationValueHB( 0.0, 'bottom', 'D270' ),
        ]

### 3.4 熱橋長さ$l_{\psi 2v,w}$

$$
l_{\psi 2v,w,D0,i,j} = \left \{
    \begin{array}{ll}
        0 & ( n_i = 1 ) \\
        H_i & ( n_i \gt 1 \  and \  j = 1 ) \\
        H_i & ( n_i \gt 1 \  and \  j = n ) \\
        H_i \times 2 & ( n_i \gt 1 , \  j \ne 1 \  and \  j \ne n ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2v,w,D90,i,j} = \left \{
    \begin{array}{ll}
        H_i \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( j = 1 ) \\
        0 & ( j \ne 1 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2v,w,D180,i,j} = \left \{
    \begin{array}{ll}
        0 & ( n_i = 1 ) \\
        H_i & ( n_i \gt 1 \  and \  j = 1 ) \\
        H_i & ( n_i \gt 1 \  and \  j = n ) \\
        H_i \times 2 & ( n_i \gt 1 , \  j \ne 1 \  and \  j \ne n ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2v,w,D270,i,j} = \left \{
    \begin{array}{ll}
        H_i \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( j = n ) \\
        0 & ( j \ne n ) \\
    \end{array}
\right.
$$

In [28]:
def get_l_psi_2v_w( l_short_total, H, n, idx ):
    return [
        OrientationValueHB( 0.0 if n == 1 else ( H if idx == 0 or idx == n - 1 else H * 2 ),   'D0',   'D0'   ),
        OrientationValueHB( H * max( l_short_total - 10, 0.0 ) / 10.0 if idx == 0 else 0.0,    'D90',  'D90'  ),
        OrientationValueHB( 0.0 if n == 1 else ( H if idx == 0 or idx == n - 1 else H * 2 ),   'D180', 'D180' ),
        OrientationValueHB( H * max( l_short_total - 10, 0.0 ) / 10.0 if idx == n-1 else 0.0,  'D270', 'D270' ),
    ]

### 3.5 熱橋長さ$l_{\psi 2h,r}$

#### i) 最上階以外($i \ne N$)の場合でかつ屋根がある($A_{r,total,i} \gt 0$)場合

$$
l_{\psi 2h,r,top,i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{r,total,i} }{ l_{long,total,i} } - 10, \  0 \right) }{ 10 } & ( n_i = 1 ) \\
        \frac{ A_{r,total,i} }{ l_{long,total,i} } + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{r,total,i} }{ l_{long,total,i} } - 10, \  0 \right) }{ 10 } & ( n_i > 1 \  and \  j = 1 ) \\
        \frac{ A_{r,total,i} }{ l_{long,total,i} } + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{r,total,i} }{ l_{long,total,i} } - 10, \  0 \right) }{ 10 } & ( n_i > 1 \  and \  j = n ) \\
        \frac{ A_{r,total,i} }{ l_{long,total,i} } \times 2 + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{r,total,i} }{ l_{long,total,i} } - 10, \  0 \right) }{ 10 } & ( n_i > 1 , \  j \ne 1 \  and \ j \ne n ) \\
    \end{array}
\right.
$$

#### ii) 最上階以外($i \ne N$)の場合でかつ屋根がない($A_{r,total,i} = 0$)場合

$$
l_{\psi 2h,r,top,i,j} = 0
$$

#### iii) 最上階($i = N$)の場合

$$
l_{\psi 2h,r,top,i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i = 1 ) \\
        l_{short,total,i} + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right)}{ 10 } & ( n_i > 1 \  and \  j = 1 ) \\
        l_{short,total,i} + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i > 1 \  and \  j = n ) \\
        l_{short,total,i} \times 2 + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i > 1 , \  j \ne 1 \  and \ k \ne n ) \\
    \end{array}
\right.
$$

In [29]:
def get_l_psi_2h_r( A_r_total, l_long_total, l_short_total, n, idx, level, levelNumber ):
    if level == levelNumber-1:
        return [
            OrientationValueHB( 0.0 if n == 1 else ( l_short_total if idx == 0 or idx == n - 1 else l_short_total * 2 )
                                 + l_long_total / n * max( l_short_total - 10.0, 0.0 ) / 10.0, 'top', 'top' )
        ]
    else:
        if A_r_total == 0.0:
            return [ OrientationValueHB( 0.0, 'top', 'top' ) ]
        else:
            return [
                OrientationValueHB(
                0.0 if n == 1 else ( A_r_total / l_long_total if idx == 0 or idx == n - 1 else A_r_total / l_long_total * 2 )
                + l_long_total / n * max( A_r_total / l_long_total - 10.0, 0.0 ) / 10.0, 'top', 'top' )
            ]

### 3.6 熱橋長さ$l_{\psi 2h,w}$

#### i) 最下階($i = 1$)の場合

$$
l_{\psi 2 h,w,D0, i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } & ( A_{of,total,i+1} = 0 ) \\
        0 & ( A_{of,total,i+1} > 0 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2 h, w, D90,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} - \frac{ A_{r,total,i} }{ l_{long,total,i} } + 1 & ( j = 0 ) \\
        1 & ( j \ne 0 ) \\
    \end{array}
\right.
$$

$$
l_{ \psi 2 h, w, D180,i,j } = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 2 h, w, D270,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} - \frac{ A_{r,total,i} }{ l_{long,total,i} } + 1 & ( j = n_i ) \\
        1 & ( j \ne n_i ) \\
    \end{array}
\right.
$$

#### ii) 最上階($i = N$)の場合

$$
l_{\psi 2 h,w,D0, i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } & ( A_{r,total,i-1} = 0 ) \\
        0 & ( A_{r,total,i-1} > 0 ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2 h, w, D90,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} - \frac{ A_{of,total,i} }{ l_{long,total,i} } + 1 & ( j = 0 ) \\
        1 & ( j \ne 0 ) \\
    \end{array}
\right.
$$

$$
l_{ \psi 2 h, w, D180,i,j } = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 2 h, w, D270,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} - \frac{ A_{of,total,i} }{ l_{long,total,i} } + 1 & ( j = n_i ) \\
        1 & ( j \ne n_i ) \\
    \end{array}
\right.
$$

#### iii) 中間階($i \ne 0 \  and \  i \ne N$)の場合

$$
l_{\psi 2 h,w,D0, i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } \times 2 & ( ( A_{of,total,i+1} = 0 ) \  and \  ( A_{r,total,i-1} = 0 ) ) \\
        \frac{ l_{long,total,i} }{ n_i } & ( ( A_{of,total,i+1} > 0 ) \  and \  ( A_{r,total,i-1} = 0 ) ) \\
        \frac{ l_{long,total,i} }{ n_i } & ( ( A_{of,total,i+1} = 0 ) \  and \  ( A_{r,total,i-1} > 0 ) ) \\
        0 & ( ( A_{of,total,i+1} > 0 ) \  and \  ( A_{r,total,i-1} > 0 ) ) \\
    \end{array}
\right.
$$

$$
l_{\psi 2 h, w, D90,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} \times 2 - \frac{ A_{r,total,i} }{ l_{long,total,i} } - \frac{ A_{of,total,i} }{ l_{long,total,i} } + 2 & ( j = 0 ) \\
        2 & ( j \ne 0 ) \\
    \end{array}
\right.
$$

$$
l_{ \psi 2 h, w, D180,i,j } = \frac{ l_{long,total,i} }{ n_i } \times 2
$$

$$
l_{\psi 2 h, w, D270,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} \times 2 - \frac{ A_{r,total,i} }{ l_{long,total,i} } - \frac{ A_{of,total,i} }{ l_{long,total,i} } + 2 & ( j = n_i ) \\
        2 & ( j \ne n_i ) \\
    \end{array}
\right.
$$

In [30]:
def get_l_psi_2h_w( A_r_total, A_r_total_mns, A_of_total, A_of_total_pls, l_long_total, l_short_total, n, idx, level, levelNumber ):
    if level == 0:
        return [
            OrientationValueHB( l_long_total / n if A_of_total_pls == 0.0 else 0.0,                     'D0',   'D0'    ),
            # ここの条件は、A_r_total == 0.0 も加えるべきではないのか？ 2018.12.10 三浦
            OrientationValueHB( l_short_total - A_r_total / l_long_total + 1.0 if idx == 0 else 1.0,    'D90',  'D90'   ),
            OrientationValueHB( l_long_total / n,                                                        'D180', 'D180' ),
            OrientationValueHB( l_short_total - A_r_total / l_long_total + 1.0 if idx == n - 1 else 1.0, 'D270', 'D270' )
        ]
    elif level == levelNumber-1:
        return [
            OrientationValueHB( l_long_total / n if A_r_total_mns == 0.0 else 0.0,                        'D0',   'D0'   ),
            # ここの条件は、A_of_tltal == 0.0 も加えるべきではないのか？ 2018.12.10 三浦
            OrientationValueHB( l_short_total - A_of_total / l_long_total + 1.0 if idx == 0 else 1.0,     'D90',  'D90'  ),
            OrientationValueHB( l_long_total / n,                                                         'D180', 'D180' ),
            OrientationValueHB( l_short_total - A_of_total / l_long_total + 1.0 if idx == n - 1 else 1.0, 'D270', 'D270' )
        ]
    else:
        return [
            OrientationValueHB( l_long_total / n * 2 if A_of_total_pls == 0.0 and A_r_total_mns == 0.0
                               else ( 0.0 if A_of_total_pls > 0.0 and A_r_total_mns > 0.0 else l_long_total / n ), 'D0', 'D0' ),
            # ここも上記と同じ疑問 2018.12.10 三浦
            OrientationValueHB( l_short_total * 2 - A_r_total / l_long_total - A_of_total / l_long_total + 2.0 if idx == 0
                               else 2.0, 'D90', 'D90' ),
            OrientationValueHB( l_long_total / n * 2, 'D180', 'D180' ),
            OrientationValueHB( l_short_total * 2 - A_r_total / l_long_total - A_of_total / l_long_total + 2.0 if idx == n - 1
                               else 2.0, 'D270', 'D270' )
        ]

### 3.7 熱橋長さ$l_{\psi 2h,f}$

#### i) 外気床がある($A_{of,total,i} \gt 0$)場合

$$
l_{\psi 2h,f,bottom,i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{of,total,i} }{ l_{long,total,i} } - 10.0, \  0.0 \right) }{ 10 } & ( n_i = 1 ) \\
        \frac{ A_{of,total,i} }{ l_{long,total,i} } + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{of,total,i} }{ l_{long,total,i} } - 10.0, \  0.0 \right) }{ 10 } & ( n_i > 1 \  and \  j = 1 ) \\
        \frac{ A_{of,total,i} }{ l_{long,total,i} } + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{of,total,i} }{ l_{long,total,i} } - 10.0, \  0.0 \right) }{ 10 } & ( n_i > 1 \  and \  j = n ) \\
        \frac{ A_{of,total,i} }{ l_{long,total,i} } \times 2 + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( \frac{ A_{of,total,i} }{ l_{long,total,i} } - 10.0, \  0.0 \right) }{ 10 } & ( n_i > 1 , \  j \ne 1 \  and \ j \ne n ) \\
    \end{array}
\right.
$$

#### ii) 外気床がない($A_{of,total,i} = 0$)場合

$$
l_{\psi 2h,f,bottom,i,j} = 0
$$

In [31]:
def get_l_psi_2h_f( A_of_total, l_long_total, n, idx ):
    if A_of_total > 0:
        if n == 1:
            return [ OrientationValueHB( l_long_total / n * max( A_of_total / l_long_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom' ) ]
        else:
            if idx == 0 or idx == n - 1:
                return [ OrientationValueHB( A_of_total / l_long_total
                                          + l_long_total / n * max( A_of_total / l_long_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom' ) ]
            else:
                return [ OrientationValueHB( A_of_total / l_long_total * 2
                                          + l_long_total / n * max( A_of_total / l_long_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom' ) ]
    else:
        return [ OrientationValueHB( 0.0, 'bottom', 'bottom' ) ]

### 3.8 熱橋長さ$l_{\psi 3v,w-w}$

$$
l_{\psi 3 v, w-w, D90-D180,i,j} = H_i \times 2
$$

$$
l_{\psi 3 v, w-w, D180-D270,i,j} = H_i \times 2
$$

In [32]:
def get_l_psi_3v_ww( H ):
    return [ OrientationValueHB( H * 2, 'D90',  'D180' ),
             OrientationValueHB( H * 2, 'D180', 'D270' ) ]

### 3.9 熱橋長さ$l_{\psi 3h,w-r}$

#### i) 最下階($i = 1$)の場合

$$
l_{\psi 3 h, w-r, top-D0,i,j} = \left \{
    \begin{array}{ll}
        0 & (A_{r,total,i}=0) \\
        \frac{ l_{long,total,i} }{ n_i } & (A_{r,total,i}>0) \\
    \end{array}
\right.
$$

#### ii) 最上階($i=N$)の場合

$$
l_{\psi 3 h, w-r, top-D0,i,j} = \left \{
    \begin{array}{ll}
        0 & (A_{r,total,i-1}=0) \\
        \frac{ l_{long,total,i} }{ n_i } & (A_{r,total,i-1}>0) \\
    \end{array}
\right.
$$

#### iii) 中間階($(i \ne 1) \  and \  (i \ne N)$)の場合

$$
l_{\psi 3 h, w-r, top-D0,i,j} = \left \{
    \begin{array}{ll}
        0 & ((A_{r,total,i}=0)\  and \  (A_{r,total,i-1}=0)) \\
        \frac{ l_{long,total,i} }{ n_i } & ((A_{r,total,i}>0)\  and \  (A_{r,total,i-1}=0)) \\
        \frac{ l_{long,total,i} }{ n_i } & ((A_{r,total,i}=0)\  and \  (A_{r,total,i-1}>0)) \\
        \frac{ l_{long,total,i} }{ n_i } \times 2 & ((A_{r,total,i}>0)\  and \  (A_{r,total,i-1}>0)) \\
    \end{array}
\right.
$$

In [33]:
def get_l_psi_3h_wr( l_long_total, A_r_total, A_r_total_mns, n, level, levelNumber ):
    if level == 0:
        return [ OrientationValueHB( 0.0 if A_r_total == 0.0 else l_long_total / n, 'top', 'D0' ) ]
    elif level == levelNumber-1:
        return [ OrientationValueHB( 0.0 if A_r_total_mns == 0.0 else l_long_total / n, 'top', 'D0' ) ]
    else:
        return [ OrientationValueHB( 0.0 if A_r_total == 0.0 and A_r_total_mns == 0.0 
                                   else ( l_long_total / n * 2 if A_r_total > 0.0 and A_r_total_mns > 0.0 else l_long_total / n ),
                                   'top', 'D0' ) ] # ここの書き方は明示的でないので直すこと

### 3.10 熱橋長さ$l_{\psi 3h,w-f}$

#### i) 最上階($i=N$)の場合

$$
l_{\psi 3 h, w-f, bottom-D0,i,j} = \left \{
    \begin{array}{ll}
        0 & (A_{of,total,i}=0) \\
        \frac{ l_{long,total,i} }{ n_i } & (A_{of,total,i}>0) \\
    \end{array}
\right.
$$

#### ii) 最上階以外($i \ne N$)の場合

$$
l_{\psi 3 h, w-f, bottom-D0,i,j} = \left \{
    \begin{array}{ll}
        0 & ((A_{of,total,i}=0)\  and \  (A_{of,total,i+1}=0)) \\
        \frac{ l_{long,total,i} }{ n_i } & ((A_{of,total,i}>0)\  and \  (A_{of,total,i+1}=0)) \\
        \frac{ l_{long,total,i} }{ n_i } & ((A_{of,total,i}=0)\  and \  (A_{of,total,i+1}>0)) \\
        \frac{ l_{long,total,i} }{ n_i } \times 2 & ((A_{of,total,i}>0)\  and \  (A_{of,total,i+1}>0)) \\
    \end{array}
\right.
$$

In [34]:
def get_l_psi_3h_wf( A_of_total, A_of_total_pls, level, levelNumber ):
    if level == levelNumber:
        return [ OrientationValueHB( 0.0 if A_of_total == 0.0 else l_long_total / n, 'bottom', 'D0' ) ]
    else:
        return [ OrientationValueHB( 0.0 if A_of_total == 0.0 and A_of_total_pls == 0.0
                                   else ( l_long_total / n * 2 if A_of_total > 0.0 and A_of_total_pls > 0.0 else l_long_total / n ),
                                   'bottom', 'D0' ) ] # ここも少し書き直した方が良い

### 3.11 熱橋長さ$l_{\psi 1h,w-uf}$

$$
l_{\psi 1 h, w-uf, bottom-D0,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1 h, w-uf,bottom-D90,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} + 1 & (j=1) \\
        1 & (j \ne 1) \\
    \end{array}
\right.
$$

$$
l_{\psi 1 h, w-uf, bottom-D180,i,j} = \frac{ l_{long,total,i} }{ n_i }
$$

$$
l_{\psi 1 h, w-uf,bottom-D270,i,j} = \left \{
    \begin{array}{ll}
        l_{short,total,i} + 1 & (j=n_i) \\
        1 & (j \ne n_i) \\
    \end{array}
\right.
$$

In [35]:
def get_l_psi_1h_wuf( l_long_total, l_short_total, idx, n ):
    return [
        OrientationValueHB( l_long_total / n, 'bottom', 'D0' ),
        OrientationValueHB( l_short_total + 1.0 if idx == 0 else 1.0, 'bottom', 'D90' ),
        OrientationValueHB( l_long_total / n, 'bottom', 'D180' ),
        OrientationValueHB( l_short_total + 1.0 if idx == 0 else n - 1, 'bottom', 'D270' ),
    ]

### 3.12 熱橋長さ$l_{\psi 2h,uf}$

#### i) 最下階($i=1$)の場合

$$
l_{\psi 2 h,uf,bottom,i,j} = \left \{
    \begin{array}{ll}
        \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i = 1 ) \\
        l_{short,total,i} + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i > 1 \  and \  j = 1 ) \\
        l_{short,total,i} + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i > 1 \  and \  j = n_i ) \\
        l_{short,total,i} \times 2 + \frac{ l_{long,total,i} }{ n_i } \times \frac{ \max \left( l_{short,total,i} - 10, \  0 \right) }{ 10 } & ( n_i > 1 , \  j \ne 1 \  and \ j \ne n_i ) \\
    \end{array}
\right.
$$

#### ii) 最下階($i \ne 1$)の場合

$$
l_{\psi 2 h,uf,bottom,i,j} = 0
$$

In [36]:
def get_l_psi_2h_uf( l_long_total, l_short_total, n, idx, level ):
    if level == 0:
        if n == 1:
            return [
                OrientationValueHB( l_long_total / n * max( l_short_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom' )
            ]
        else:
            if idx != 0 and idx != n - 1:
                return [
                    OrientationValueHB( l_short_total *  2 + l_long_total / n * max( l_short_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom')
                ]
            else:
                return [
                    OrientationValueHB( l_short_total + l_long_total / n * max( l_short_total - 10.0, 0.0 ) / 10.0, 'bottom', 'bottom')
                ]
    else:
        return [ OrientationValueHB( 0.0, 'bottom', 'bottom' ) ]

## V. クラス

In [37]:
class Building():
    
    def __init__(self, d):
        
        common = d['common']
        
        self.region       = common['region']
        self.structure    = common['structure']
        self.levels       = common['levels_number']
        self.height       = common['height']
        self.direction    = common['opening_main_direction']
        
        self.private_area = PrivateArea(d, self.levels)
    
    def calc(self):
        
        self.private_area.calc(self)
    
    def get_dict(self):
        
        return {
            'common' : {
                'region'    : self.region,
                'structure' : self.structure,
            },
            'private_area' : self.private_area.get_dict()
        }

In [38]:
class PrivateArea():
    
    def __init__(self, d, levels_number):
        
        private_area = d['private_area']
        
        self.wall_U_value         = private_area['wall']['U_value']
        self.wall_ins_pos         = private_area['wall']['ins_pos']
        self.wall_HB_spec         = private_area['wall']['HB_spec']
        self.roof_U_value         = private_area['roof']['U_value']
        self.roof_ins_pos         = private_area['roof']['ins_pos']
        self.roof_HB_spec         = private_area['roof']['HB_spec']
        self.floor_U_value        = private_area['floor']['U_value']
        self.floor_ins_pos        = private_area['floor']['ins_pos']
        self.floor_HB_spec        = private_area['floor']['HB_spec']
        self.inside_wall_U_value  = private_area['inside_wall']['U_value']
        self.inside_floor_U_value = private_area['inside_floor']['U_value']
        self.window_U_value       = private_area['window']['U_value']
        self.window_eta_value     = private_area['window']['eta_value']
        
        self.floors = [ PrivateFloor(level, d) for level in range(levels_number) ]
    
    def calc(self, building):
        
        for f in self.floors:
            f.calc(building, self)
            
    def get_dict(self):
        
        return {
            'wall' : {
                'U_value' : self.wall_U_value,
                'ins_pos' : self.wall_ins_pos,
                'HB_spec' : self.wall_HB_spec,
            },
            'roof' : {
                'U_value' : self.roof_U_value,
                'ins_pos' : self.roof_ins_pos,
                'HB_spec' : self.roof_HB_spec,
            },
            'floor' : {
                'U_value' : self.floor_U_value,
                'ins_pos' : self.floor_ins_pos,
                'HB_spec' : self.floor_HB_spec,
            },
            'inside_wall' : {
                'U_value' : self.inside_wall_U_value,
            },
            'inside_floor' : {
                'U_value' : self.inside_floor_U_value,
            },
            'window' : {
                'U_value' : self.window_U_value,
                'eta_value' : self.window_eta_value,
            },
            'private_floors' : [f.get_dict() for f in self.floors]
        }

In [39]:
class PrivateFloor():
    
    def __init__(self, level, d):
        
        self.level = level
        
        private_floor = d['private_area']['private_floors'][self.level]

        self.house_number             = private_floor['house_number']
        self.floor_area               = private_floor['floor_area']
        self.peri_length_outside      = private_floor['peri_length_outside']
        self.window_area              = private_floor['window_area']
        self.roof_area_total_input    = private_floor['roof_area']
        self.outside_floor_area_input = private_floor['outside_floor_area']
        
        levels_number = d['common']['levels_number']
        self.roof_area_total = get_roof_area_total(
            self.roof_area_total_input, self.floor_area, self.level, levels_number) # 屋根面積
        self.outside_floor_area_total = get_outside_floor_area_total(self.outside_floor_area_input, self.level) # 外気に接する床の面積
        
        self.houses = [ House(i) for i in range(self.house_number) ]

    def calc(self, building, private_area):
        
        self.height = get_height(building.height, building.levels) # 階高
        
        self.l_long_total  = get_l_long_total(self.peri_length_outside, self.floor_area) # 長辺の長さ
        self.l_short_total = get_l_short_total(self.peri_length_outside, self.floor_area) # 短辺の長さ
        
        self.windows_area_total = get_windows_area_total(self.window_area, self.house_number) # 窓面積の合計
        
        for h in self.houses:
            h.calc(building, self)
        
    def get_dict(self):
        
        return {
            'level'  : self.level,
            'height' : self.height,
            'houses' : [ h.get_dict() for h in self.houses ]
        }

In [40]:
class House():
    
    def __init__(self, index):
        
        self.index = index
    
    def calc(self, building, floor):
        
        self.roof_area = get_roof_area(
            floor.roof_area_total, floor.house_number) # 屋根面積
        
        self.windows_area = get_windows_area(
            floor.windows_area_total, floor.house_number, self.index) # 窓面積
        
        self.doors_area = get_doors_area() # 玄関ドア面積
        
        self.outside_walls_area = get_outside_walls_area(
            floor.l_long_total, floor.l_short_total, floor.height, self.windows_area,
            self.doors_area, floor.house_number, self.index) # 外壁面積
        
        self.outside_floor_area = get_outside_floor_area(
            floor.outside_floor_area_total, floor.house_number) # 外気に面する床面積

        self.inside_wall_area = get_inside_wall_area(
            floor.l_short_total, floor.height, floor.house_number, self.index) # 界壁面積
        
        self.inside_floor_area = get_inside_floor_area(
            floor.floor_area, self.outside_floor_area, floor.house_number, floor.level) # 界床面積

        self.inside_ceiling_area = get_inside_ceiling_area(
            floor.floor_area, self.roof_area, floor.house_number, floor.level, building.levels) # 界天井面積
        
        self.crawl_space_floor_area = get_crawl_space_floor_area(
            floor.floor_area, floor.house_number, floor.level) # 床下に面する床面積
        
        # 熱橋長さ
        self.l_psi_1v_ww = get_l_psi_1v_ww(
            floor.height, floor.house_number, self.index)
        self.l_psi_1h_wr = get_l_psi_1h_wr(
            floor.l_long_total, floor.l_short_total, floor.roof_area_total, floor.house_number, self.index,
            floor.level, building.levels)
        self.l_psi_1h_wf = get_l_psi_1h_wf(
            floor.l_long_total, floor.outside_floor_area_total, floor.house_number, self.index)
        self.l_psi_2v_w = get_l_psi_2v_w(
            floor.l_short_total, floor.height, floor.house_number, self.index)
        self.l_psi_2h_r = get_l_psi_2h_r(
            floor.roof_area_total, floor.l_long_total, floor.l_short_total, floor.house_number,
            self.index, floor.level, building.levels)
        
        # １つ上の階の外気床の面積 もっと良い書き方があるかもしれないので後日書き換えること
        A_of_total_pls = 0.0 if floor.level == building.levels - 1 else building.private_area.floors[floor.level+1].outside_floor_area_total
        # 1つ下の階の屋根の面積 もっと良い書き方があるかもしれないので後日書き換えること
        A_r_total_mns  = 0.0 if floor.level == 0 else building.private_area.floors[floor.level-1].roof_area_total
        # ↑１つ上の階層の、floorで書いた方が良い。
        
        self.l_psi_2h_w = get_l_psi_2h_w(
            floor.roof_area_total, A_r_total_mns, floor.outside_floor_area_total, A_of_total_pls,
            floor.l_long_total, floor.l_short_total, floor.house_number, self.index, floor.level, building.levels )
        self.l_psi_2h_f   = get_l_psi_2h_f(
            floor.outside_floor_area_total, floor.l_long_total, floor.house_number, self.index )
        self.l_psi_3v_ww = get_l_psi_3v_ww(floor.height)
        self.l_psi_3h_wr = get_l_psi_3h_wr(
            floor.l_long_total, floor.roof_area_total, A_r_total_mns, floor.house_number, floor.level, building.levels)
        self.l_psi_3h_wf = get_l_psi_3h_wf(
            floor.outside_floor_area_total, A_of_total_pls, floor.level, building.levels)
        self.l_psi_1h_wuf = get_l_psi_1h_wuf(
            floor.l_long_total, floor.l_short_total, self.index, floor.house_number)
        self.l_psi_2h_uf = get_l_psi_2h_uf(
            floor.l_long_total, floor.l_short_total, floor.house_number, self.index, floor.level)

    def get_dict(self):
        
        return {
            'index'                  : self.index,
            'roof_area'              : self.roof_area,
            'windows_area'           : self.windows_area.get_dict(),
            'doors_area'             : self.doors_area.get_dict(),
            'outside_walls_area'     : self.outside_walls_area.get_dict(),
            'outside_floor_area'     : self.outside_floor_area,
            'inside_wall_area'       : self.inside_wall_area,
            'inside_floor_area'      : self.inside_floor_area,
            'inside_ceiling_area'    : self.inside_ceiling_area,
            'crawl_space_floor_area' : self.crawl_space_floor_area,
            'heat_bridge_length' : {
                '1v_ww'  : [l.get_info() for l in self.l_psi_1v_ww],
                '1h_wr'  : [l.get_info() for l in self.l_psi_1h_wr],
                '1h_wf'  : [l.get_info() for l in self.l_psi_1h_wf],
                '2v_w'   : [l.get_info() for l in self.l_psi_2v_w],
                '2h_r'   : [l.get_info() for l in self.l_psi_2h_r],
                '2h_w'   : [l.get_info() for l in self.l_psi_2h_w],
                '2h_f'   : [l.get_info() for l in self.l_psi_2h_f],
                '3v_ww'  : [l.get_info() for l in self.l_psi_3v_ww],
                '3h_wr'  : [l.get_info() for l in self.l_psi_3h_wr],
                '3h_wf'  : [l.get_info() for l in self.l_psi_3h_wf],
                '1h_wuf' : [l.get_info() for l in self.l_psi_1h_wuf],
                '2h_uf'  : [l.get_info() for l in self.l_psi_2h_uf],
            },
        }

In [41]:
d = {
    'common' : {
        'region'                 : 6,    # 地域の区分
        'structure'              : 'RC', # 建物の構造
        'levels_number'          : 3,    # 建物の階数
        'height'                 : 9.1,  # 建物の高さ(m)
        'opening_main_direction' : 'S',  # 主開口部の方位
    },
    'private_area' : {
        'private_floors' : [    # 各階の情報
            {
                'house_number'        : 4,         # 住戸数
                'floor_area'          : 162.64,    # 床面積の合計(m2)
                'peri_length_outside' : 66.56,     # 外周長 外気と接する部分(m)
                'PeriLengthInside'    : 0.0,       # 外周長 空調された共用部や非住宅部と接する部分(m)
                'PeriLengthPrivate'   : 0.0,       # 外周長 住戸と接する部分(住棟を複数の方位に分けて計算する場合のみ)(m)
                'window_area'         : 30.48,     # 窓面積の合計(m2)
                'roof_area'           : 0.0,       # (最上階を除く)屋根面積の合計(m2)
                'outside_floor_area'  : None,      # (最下階を除く)外気床の面積の合計(m2)
            },
            {
                'house_number'        : 4,         # 住戸数
                'floor_area'          : 162.64,    # 床面積の合計(m2)
                'peri_length_outside' : 66.56,     # 外周長 外気と接する部分(m)
                'PeriLengthInside'    : 0.0,       # 外周長 空調された共用部や非住宅部と接する部分(m)
                'PeriLengthPrivate'   : 0.0,       # 外周長 住戸と接する部分(住棟を複数の方位に分けて計算する場合のみ)(m)
                'window_area'         : 30.64,     # 窓面積の合計(m2)
                'roof_area'           : 0.0,       # (最上階を除く)屋根面積の合計(m2)
                'outside_floor_area'  : 0.0,       # (最下階を除く)外気床の面積の合計(m2)
            },
            {
                'house_number'        : 4,         # 住戸数
                'floor_area'          : 162.64,    # 床面積の合計(m2)
                'peri_length_outside' : 66.56,     # 外周長 外気と接する部分(m)
                'PeriLengthInside'    : 0.0,       # 外周長 空調された共用部や非住宅部と接する部分(m)
                'PeriLengthPrivate'   : 0.0,       # 外周長 住戸と接する部分(住棟を複数の方位に分けて計算する場合のみ)(m)
                'window_area'         : 30.64,     # 窓面積の合計(m2)
                'roof_area'           : None,      # (最上階を除く)屋根面積の合計(m2)
                'outside_floor_area'  : 0.0,       # (最下階を除く)外気床の面積の合計(m2)
            },
        ],
        'wall' : {                 # 外壁の仕様
            'U_value' : 0.785,     # 熱貫流率(U値)(W/m2K)
            'ins_pos' : 'inside',  # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
            'HB_spec' : 'spec1'    # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
        },
        'roof' : {                 # 屋根の仕様
            'U_value' : 0.417,     # 熱貫流率(U値)(W/m2K)
            'ins_pos' : 'outside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
            'HB_spec' : 'spec1'    # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
        },
        'floor' : {                # 最下階床の仕様
            'U_value' : 0.507,     # 熱貫流率(U値)(W/m2K)
            'ins_pos' : 'outside', # 断熱位置, inside=内断熱, outside=外断熱, non=無断熱 から選択
            'HB_spec' : 'null'     # 断熱補強の仕様, spec1=仕様1, spec2=仕様2, non=補強なし, null=該当なし から選択
        },
        'inside_wall' : {
            'U_value'   : 2.34, # 界壁の熱貫流率(U値)(W/m2K)
        },
        'inside_floor' : {
            'U_value'  : 1.796, # 界床の熱貫流率(U値)(W/m2K)
        },
        'window' : {
            'U_value'   : 4.65, # 窓の熱貫流率(U値)(W/m2K)
            'eta_value' : 0.79, # 窓の日射熱取得率(η値)    
        }
    },
    "PublicArea" : {
        'FloorArea'          : [68.82, 59.7, 59.7], # 各階の共用部・非住宅部の床面積の合計(m2)
        'RoofArea'           : [9.125, 0.0, None],  # 各階(最上階を除く)の屋根面積の合計(m2)
        'OutsideFloorArea'   : [None, 0.0, 0.0],    # 各階(最下階を除く)の外気床の面積の合計(m2)
        'WallUValue'         : 0.00,                # 外壁の熱貫流率(U値)(W/m2K)
        'InsideFloorUValue'  : 0.00,                # 界床の熱貫流率(U値)(W/m2K)
        'OutsideFloorUValue' : 0.00,                # 外気床の熱貫流率(U値)(W/m2K)
        'WindowUValue'       : 0.00,                # 窓の熱貫流率(U値)(W/m2K)
        'WindowEtaValue'     : 0.00,                # 窓の日射熱取得率(η値)
    },
}

In [42]:
b = Building(d)
b.calc()

pprint.pprint(b.get_dict(), compact=True)

{'common': {'region': 6, 'structure': 'RC'},
 'private_area': {'floor': {'HB_spec': 'null',
                            'U_value': 0.507,
                            'ins_pos': 'outside'},
                  'inside_floor': {'U_value': 1.796},
                  'inside_wall': {'U_value': 2.34},
                  'private_floors': [{'height': 3.033333333333333,
                                      'houses': [{'crawl_space_floor_area': 40.66,
                                                  'doors_area': {'D0': 0.0,
                                                                 'D180': 1.6,
                                                                 'D270': 0.0,
                                                                 'D90': 0.0},
                                                  'heat_bridge_length': {'1h_wf': [{'orientation1': 'bottom',
                                                                                    'orientation2': 'D0',
                             

                                                                         '1h_wr': [{'orientation1': 'top',
                                                                                    'orientation2': 'D0',
                                                                                    'value': 0.0},
                                                                                   {'orientation1': 'top',
                                                                                    'orientation2': 'D90',
                                                                                    'value': 0.0},
                                                                                   {'orientation1': 'top',
                                                                                    'orientation2': 'D180',
                                                                                    'value': 0.0},
                                                             

                                                                                    'value': 0.0}],
                                                                         '2h_w': [{'orientation1': 'D0',
                                                                                   'orientation2': 'D0',
                                                                                   'value': 13.664380225994407},
                                                                                  {'orientation1': 'D90',
                                                                                   'orientation2': 'D90',
                                                                                   'value': 2.0},
                                                                                  {'orientation1': 'D180',
                                                                                   'orientation2': 'D180',
                                                     

                                                  'outside_walls_area': {'D0': 16.11605400942485,
                                                                         'D180': 16.53216600942485,
                                                                         'D270': 3.033333333333333,
                                                                         'D90': 3.033333333333333},
                                                  'roof_area': 40.66,
                                                  'windows_area': {'D0': 4.608256,
                                                                   'D180': 2.592144,
                                                                   'D270': 0,
                                                                   'D90': 0}},
                                                 {'crawl_space_floor_area': 40.66,
                                                  'doors_area': {'D0': 0.0,
                                                